In [ ]:
!apt update

In [ ]:
!apt-get install git-all -y

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git

In [ ]:
!pwd

In [ ]:
%cd YOLOX

In [ ]:
!pip install -U pip && pip install -r requirements.txt
!pip install opencv-python --upgrade

In [9]:
!mkdir weights

In [ ]:
%cd weights/

In [ ]:
!apt-get install wget -y

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth

In [ ]:
%cd -

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!python3 demo.py \
image -f exps/example/custom/yolox_s.py \
-c "./weights/yolox_s.pth" \
--path "./data" \
--conf 0.3 \
--nms 0.45 \
--tsize 640 \
--save_result

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install onnx

In [ ]:
!pip install onnxruntime-gpu

## Convert pytorch to onnx

In [ ]:
!python export_onnx.py --output-name yolox_s.onnx -n yolox-s -c ./weights/yolox_s.pth

# -n：ベースとなるモデル名。[yolox-s、m、l、xおよびyolox-nane、yolox-tiny、yolov3]から選択。
# -c：変換前のモデル名
# --output-name：変換後のモデル名

In [47]:
import argparse
import os
import cv2
import numpy as np
import pandas as pd
import onnxruntime
from yolox.data.data_augment import preproc as preprocess
from yolox.data.datasets import COCO_CLASSES
from yolox.utils import mkdir, multiclass_nms, demo_postprocess, vis


output_dir ='onnx_out'
image_path = './data/48_273271149030004599ea82.jpg'
model = './yolox_s.onnx'

input_shape = (640,640)


origin_img = cv2.imread(image_path)
img, ratio = preprocess(origin_img, input_shape)
session = onnxruntime.InferenceSession(model)
ort_inputs = {session.get_inputs()[0].name: img[None, :, :, :]}
output = session.run(None, ort_inputs)
predictions = demo_postprocess(output[0], input_shape)[0]
boxes = predictions[:, :4]
scores = predictions[:, 4:5] * predictions[:, 5:]
boxes_xyxy = np.ones_like(boxes)
boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
boxes_xyxy /= ratio
dets = multiclass_nms(boxes_xyxy, scores, nms_thr=0.45, score_thr=0.5)
if dets is not None:
    final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
    origin_img = vis(origin_img, final_boxes, final_scores, final_cls_inds,
                      0.3, class_names=COCO_CLASSES)

mkdir(output_dir)
output_path = os.path.join(output_dir, os.path.basename(image_path))

cv2.imwrite(output_path, origin_img)


True

In [48]:
import zipfile
with zipfile.ZipFile('','r') as inputFile:
    inputFile.extractall()

## Tranining custom dataset

In [ ]:
!pip install protobuf==3.20.*

In [ ]:
!pip install numpy==1.23.0

In [ ]:
%cd YOLOX

In [ ]:
!python3 train.py -f exps/example/custom/yolox_s.py -d 1 -b 32 --fp16 --cache  -c ./weights/yolox_s.pth

In [ ]:
import torch
print(torch.cuda.is_available())

### Isssue

**ModuleNotFoundError: No module named 'yolox.layers.fast_cocoeval'**

### sulution

<!-- 294行目を下記に変更する -->
from pycocotools.cocoeval import COCOeval


In [ ]:
!python3 eval.py -n  yolox-s -c ./YOLOX_outputs/yolox_s/best_ckpt.pth -b 64 -d 1 --conf 0.001 -f ./exps/example/custom/yolox_s.py


In [ ]:
!python3 demo.py image -f exps/example/custom/yolox_s.py -c ./YOLOX_outputs/yolox_s/best_ckpt.pth --path ./data_ --conf 0.3 --nms 0.45 --tsize 640 --save_result


In [ ]:
!python export_onnx.py --output-name yolox_s_custom.onnx -n yolox-s -c ./YOLOX_outputs/yolox_s/best_ckpt.pth

# -n：ベースとなるモデル名。[yolox-s、m、l、xおよびyolox-nane、yolox-tiny、yolov3]から選択。
# -c：変換前のモデル名
# --output-name：変換後のモデル名

In [84]:
import argparse
import os
import cv2
import numpy as np
import pandas as pd
import onnxruntime
from yolox.data.data_augment import preproc as preprocess
from yolox.data.datasets import COCO_CLASSES
from yolox.utils import mkdir, multiclass_nms, demo_postprocess, vis


output_dir ='onnx_out'
image_path = './data_/48_273271149030004599ea82.jpg'
model = './yolox_s_custom.onnx'

input_shape = (640,640)


origin_img = cv2.imread(image_path)
img, ratio = preprocess(origin_img, input_shape)
session = onnxruntime.InferenceSession(model)
ort_inputs = {session.get_inputs()[0].name: img[None, :, :, :]}
output = session.run(None, ort_inputs)
predictions = demo_postprocess(output[0], input_shape)[0]
boxes = predictions[:, :4]
scores = predictions[:, 4:5] * predictions[:, 5:]
boxes_xyxy = np.ones_like(boxes)
boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
boxes_xyxy /= ratio
dets = multiclass_nms(boxes_xyxy, scores, nms_thr=0.45, score_thr=0.5)
if dets is not None:
    final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
    origin_img = vis(origin_img, final_boxes, final_scores, final_cls_inds,
                      0.3, class_names=COCO_CLASSES)

mkdir(output_dir)
output_path = os.path.join(output_dir, os.path.basename(image_path))

cv2.imwrite(output_path, origin_img)


True

In [10]:
import zipfile
with zipfile.ZipFile('','r') as inputFile:
    inputFile.extractall()